In [25]:
import tstrippy
import h5py

In [14]:
path_to_init_conds = "/scratch2/sferrone/simulations/MonteCarloObservables/Pal5-observables.hdf5"
path_to_init_conds

'/scratch2/sferrone/simulations/MonteCarloObservables/Pal5-observables.hdf5'

In [18]:
myinit_conds=h5py.File(path_to_init_conds, 'r')

<KeysViewHDF5 ['monte-carlo-000', 'monte-carlo-001', 'monte-carlo-002', 'monte-carlo-003', 'monte-carlo-004', 'monte-carlo-005', 'monte-carlo-006', 'monte-carlo-007', 'monte-carlo-008', 'monte-carlo-009', 'monte-carlo-010', 'monte-carlo-011', 'monte-carlo-012', 'monte-carlo-013', 'monte-carlo-014', 'monte-carlo-015', 'monte-carlo-016', 'monte-carlo-017', 'monte-carlo-018', 'monte-carlo-019', 'monte-carlo-020', 'monte-carlo-021', 'monte-carlo-022', 'monte-carlo-023', 'monte-carlo-024', 'monte-carlo-025', 'monte-carlo-026', 'monte-carlo-027', 'monte-carlo-028', 'monte-carlo-029', 'monte-carlo-030', 'monte-carlo-031', 'monte-carlo-032', 'monte-carlo-033', 'monte-carlo-034', 'monte-carlo-035', 'monte-carlo-036', 'monte-carlo-037', 'monte-carlo-038', 'monte-carlo-039', 'monte-carlo-040', 'monte-carlo-041', 'monte-carlo-042', 'monte-carlo-043', 'monte-carlo-044', 'monte-carlo-045', 'monte-carlo-046', 'monte-carlo-047', 'monte-carlo-048', 'monte-carlo-049']>

In [24]:
myinit_conds['monte-carlo-000']['DEC'][0]

-0.121